<h1> Parameter Tuning
    
    Look for the best kernel initializer and bias initializer

# Importing libraries:

In [4]:
!pip install pandas
!pip install sklearn
!pip install matplotlib

import matplotlib.pyplot as plt
import pandas as pd;
import numpy as np;
import sklearn

# Import data; set X and y; fill nan values and split in test and training  data:

In [5]:
# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# for later use
features_num = 15

# data
data = data.loc[data.index > 2018070000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# fill nan values
data.fillna(method = 'ffill', inplace = True)

from sklearn.model_selection import train_test_split

# divide data into train and test 
data_train, data_test = train_test_split(
         data, test_size = 0.15, shuffle=False)

# Apply feature scaling:

In [6]:
from sklearn.preprocessing import MinMaxScaler

# data scaling  (including offer (y))
sc_X = MinMaxScaler()
data_train = sc_X.fit_transform(data_train)
data_test = sc_X.transform(data_test)

# Importing the Keras libraries and packages:

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping

mae_gen = []
rmse_gen = []
mae_nor = []
mae_spi = []
rmse_nor = []
rmse_spi = []
hist_list = []
y_pred_list = []
prediction_list = []
time_count = []

# Create regressor:

In [8]:
# parameters
steps = 96
n_hidden = 1
units = 150
batch_size = 96

from sklearn.model_selection import ParameterGrid

# Dictionary to include the parameters
parameters = {'bias_initializer':[initializers.Zeros(),
                                 initializers.Ones()],
              'kernel_initializer': ['glorot_uniform',
                                     'he_normal',
                                     'he_uniform']
               }

all_param = ParameterGrid(parameters)

# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

# divide features and labels
X_train = data_train[:, 0:15] 
y_train = data_train[:, -1]
X_test = data_test[:, 0:15] 
y_test = data_test[:, -1] 

# divide data into train and test 
X_train, X_val, y_train, y_val = train_test_split(
         X_train, y_train, test_size = 0.15, shuffle=False)

# put data into correct shape
X_train, y_train = split_data(X_train, y_train, steps)
X_test, y_test = split_data(X_test, y_test, steps)
X_val, y_val = split_data(X_val, y_val, steps)

X_train = cut_data(X_train, batch_size)
y_train = cut_data(y_train, batch_size)
X_test = cut_data(X_test, batch_size)
y_test = cut_data(y_test, batch_size)
X_val = cut_data(X_val, batch_size)
y_val = cut_data(y_val, batch_size)

# inverse of test set should not be inside the loop 
y_test = (y_test * sc_X.data_range_[15]) + (sc_X.data_min_[15])

# smal adjustment
y_test = pd.Series(y_test)
y_test.replace(0, 0.0001,inplace = True)

for i in range(len(all_param)):
    
    bias_initializer = all_param[i]['bias_initializer']
    kernel_initializer = all_param[i]['kernel_initializer']

    # design the LSTM
    def regressor_tunning(bias_initializer, kernel_initializer):
        model = Sequential()
        if n_hidden == 0:
            model.add(LSTM(units = units,                    
                           batch_input_shape = (batch_size, steps, features_num), 
                           stateful = True,
                           kernel_initializer = kernel_initializer,
                           bias_initializer = bias_initializer))
            model.add(LeakyReLU(alpha = 0.2))
            model.add(Dropout(0.2))
        else:
            model.add(LSTM(units = units,                    
                           batch_input_shape = (batch_size, steps, features_num), 
                           stateful = True,
                           return_sequences = True,
                           kernel_initializer = kernel_initializer,
                           bias_initializer = bias_initializer))
            model.add(LeakyReLU(alpha = 0.2))
            model.add(Dropout(0.2))
            model.add(LSTM(units = units, 
                           batch_input_shape = (batch_size, steps, features_num), 
                           stateful = True,
                           kernel_initializer = kernel_initializer,
                           bias_initializer = bias_initializer))
            model.add(LeakyReLU(alpha = 0.2))
            model.add(Dropout(0.2))
        model.add(Dense(1, activation='linear'))
        optimizer = optimizers.RMSprop(lr = i)
        model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = optimizer)
        return model

    model = regressor_tunning(bias_initializer, kernel_initializer)

    # fitting the LSTM to the training set
    history = model.fit(X_train,
                        y_train, 
                        batch_size = batch_size, 
                        epochs = 180,
                        shuffle = False, 
                        validation_data = (X_val, y_val))
    
    model.reset_states()
    
    # make new predicitons with test set
    y_pred = model.predict(X_test, batch_size = batch_size)
    
    # prices col = 15
    y_pred = (y_pred * sc_X.data_range_[15]) + (sc_X.data_min_[15])

    y_pred_list.append(y_pred)
    
    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mae_error = mae(y_test, y_pred)
    
    rmse_gen.append(rmse_error)
    mae_gen.append(mae_error)
    
    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])
    
    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values

    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mae_spike = mae(y_test_spike, y_pred_spike)
    
    rmse_spi.append(rmse_spike)
    mae_spi.append(mae_spike)
    
    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================
    
    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)
    
    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mae_normal = mae(y_test_normal, y_pred_normal)
    
    rmse_nor.append(rmse_normal)
    mae_nor.append(mae_normal)

Epoch 1/180
65/65 [==============================] - 29s 446ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0638 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 2/180
65/65 [==============================] - 28s 424ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0639 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 3/180
65/65 [==============================] - 28s 427ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0643 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 4/180
65/65 [==============================] - 29s 440ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0646 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 5/180
65/65 [==============================] - 29s 449ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0637 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 6/180
65/65 [==============================] - 28s 434ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0639 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoc

65/65 [==============================] - 25s 387ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0641 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 51/180
65/65 [==============================] - 27s 422ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0642 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 52/180
65/65 [==============================] - 38s 580ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0643 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 53/180
65/65 [==============================] - 25s 390ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0642 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 54/180
65/65 [==============================] - 29s 454ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0642 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 55/180
65/65 [==============================] - 28s 437ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0638 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 56/18

65/65 [==============================] - 31s 483ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0642 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 100/180
65/65 [==============================] - 25s 381ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0645 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 101/180
65/65 [==============================] - 29s 447ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0647 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 102/180
65/65 [==============================] - 30s 466ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0641 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 103/180
65/65 [==============================] - 33s 509ms/step - loss: 0.0055 - mse: 0.0055 - mae: 0.0648 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 104/180
65/65 [==============================] - 31s 473ms/step - loss: 0.0055 - mse: 0.0055 - mae: 0.0646 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 

Epoch 148/180
65/65 [==============================] - 31s 478ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0642 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 149/180
65/65 [==============================] - 32s 491ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0641 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 150/180
65/65 [==============================] - 30s 469ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0643 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 151/180
65/65 [==============================] - 26s 407ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0638 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 152/180
65/65 [==============================] - 35s 532ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0643 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0635
Epoch 153/180
65/65 [==============================] - 35s 534ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0639 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae:

65/65 [==============================] - 33s 502ms/step - loss: 514.7709 - mse: 514.7709 - mae: 22.5918 - val_loss: 496.3664 - val_mse: 496.3664 - val_mae: 22.2793
Epoch 17/180
65/65 [==============================] - 32s 487ms/step - loss: 521.5306 - mse: 521.5306 - mae: 22.6871 - val_loss: 400.5416 - val_mse: 400.5416 - val_mae: 20.0135
Epoch 18/180
65/65 [==============================] - 32s 491ms/step - loss: 511.6643 - mse: 511.6643 - mae: 22.4612 - val_loss: 606.9275 - val_mse: 606.9275 - val_mae: 24.6359
Epoch 19/180
65/65 [==============================] - 38s 590ms/step - loss: 519.8469 - mse: 519.8468 - mae: 22.6929 - val_loss: 580.4811 - val_mse: 580.4811 - val_mae: 24.0932
Epoch 20/180
65/65 [==============================] - 264s 4s/step - loss: 514.7852 - mse: 514.7852 - mae: 22.5984 - val_loss: 505.4376 - val_mse: 505.4376 - val_mae: 22.4819
Epoch 21/180
65/65 [==============================] - 28s 426ms/step - loss: 515.7129 - mse: 515.7129 - mae: 22.5893 - val_loss: 6

...

65/65 [==============================] - 40s 612ms/step - loss: 19118.3496 - mse: 19118.3496 - mae: 137.5537 - val_loss: 15502.2695 - val_mse: 15502.2695 - val_mae: 124.5077
Epoch 147/180
65/65 [==============================] - 35s 541ms/step - loss: 19190.7363 - mse: 19190.7363 - mae: 137.4263 - val_loss: 17370.9551 - val_mse: 17370.9551 - val_mae: 131.7989
Epoch 148/180
65/65 [==============================] - 34s 517ms/step - loss: 19107.3125 - mse: 19107.3125 - mae: 137.6145 - val_loss: 20756.5586 - val_mse: 20756.5586 - val_mae: 144.0702
Epoch 149/180
65/65 [==============================] - 35s 533ms/step - loss: 19315.6250 - mse: 19315.6250 - mae: 138.3320 - val_loss: 17457.5898 - val_mse: 17457.5898 - val_mae: 132.1271
Epoch 150/180
65/65 [==============================] - 36s 551ms/step - loss: 19050.0625 - mse: 19050.0625 - mae: 137.4062 - val_loss: 16678.2109 - val_mse: 16678.2129 - val_mae: 129.1414
Epoch 151/180
65/65 [==============================] - 35s 536ms/step - lo

Epoch 10/180
65/65 [==============================] - 37s 576ms/step - loss: 29855.1562 - mse: 29855.1562 - mae: 171.8941 - val_loss: 30769.8086 - val_mse: 30769.8086 - val_mae: 175.4132
Epoch 11/180
65/65 [==============================] - 37s 567ms/step - loss: 29532.8770 - mse: 29532.8770 - mae: 171.2268 - val_loss: 29469.1523 - val_mse: 29469.1523 - val_mae: 171.6658
Epoch 12/180
65/65 [==============================] - 37s 572ms/step - loss: 30024.6895 - mse: 30024.6895 - mae: 172.4089 - val_loss: 30754.9395 - val_mse: 30754.9395 - val_mae: 175.3709
Epoch 13/180
65/65 [==============================] - 39s 604ms/step - loss: 29333.6543 - mse: 29333.6543 - mae: 170.6131 - val_loss: 37584.6211 - val_mse: 37584.6211 - val_mae: 193.8675
Epoch 14/180
65/65 [==============================] - 36s 553ms/step - loss: 29717.8340 - mse: 29717.8340 - mae: 171.7136 - val_loss: 30064.6855 - val_mse: 30064.6855 - val_mae: 173.3917
Epoch 15/180
65/65 [==============================] - 34s 520ms/s

Epoch 54/180
65/65 [==============================] - 40s 608ms/step - loss: 29789.5176 - mse: 29789.5176 - mae: 171.9549 - val_loss: 31775.0371 - val_mse: 31775.0371 - val_mae: 178.2555
Epoch 55/180
65/65 [==============================] - 39s 594ms/step - loss: 29648.4023 - mse: 29648.4023 - mae: 171.4218 - val_loss: 30247.6738 - val_mse: 30247.6738 - val_mae: 173.9185
Epoch 56/180
65/65 [==============================] - 36s 552ms/step - loss: 29747.6484 - mse: 29747.6484 - mae: 171.3646 - val_loss: 29225.4707 - val_mse: 29225.4707 - val_mae: 170.9546
Epoch 57/180
65/65 [==============================] - 38s 591ms/step - loss: 29785.3438 - mse: 29785.3438 - mae: 171.6945 - val_loss: 23133.1621 - val_mse: 23133.1621 - val_mae: 152.0959
Epoch 58/180
65/65 [==============================] - 35s 539ms/step - loss: 29739.7129 - mse: 29739.7129 - mae: 171.5205 - val_loss: 26011.7246 - val_mse: 26011.7246 - val_mae: 161.2815
Epoch 59/180
65/65 [==============================] - 35s 537ms/s

Epoch 98/180
65/65 [==============================] - 37s 568ms/step - loss: 29567.5059 - mse: 29567.5059 - mae: 171.1083 - val_loss: 26330.6484 - val_mse: 26330.6484 - val_mae: 162.2672
Epoch 99/180
65/65 [==============================] - 40s 620ms/step - loss: 29713.1309 - mse: 29713.1309 - mae: 171.7309 - val_loss: 24412.2441 - val_mse: 24412.2441 - val_mae: 156.2442
Epoch 100/180
65/65 [==============================] - 36s 557ms/step - loss: 29642.8008 - mse: 29642.8047 - mae: 171.3214 - val_loss: 23296.2090 - val_mse: 23296.2090 - val_mae: 152.6310
Epoch 101/180
65/65 [==============================] - 34s 519ms/step - loss: 29612.9531 - mse: 29612.9531 - mae: 171.0905 - val_loss: 25137.5371 - val_mse: 25137.5371 - val_mae: 158.5482
Epoch 102/180
65/65 [==============================] - 33s 505ms/step - loss: 29850.5684 - mse: 29850.5684 - mae: 172.0873 - val_loss: 31645.4785 - val_mse: 31645.4785 - val_mae: 177.8918
Epoch 103/180
65/65 [==============================] - 33s 505

65/65 [==============================] - 37s 565ms/step - loss: 29824.9082 - mse: 29824.9082 - mae: 171.9530 - val_loss: 33568.0625 - val_mse: 33568.0625 - val_mae: 183.2159
Epoch 142/180
65/65 [==============================] - 35s 537ms/step - loss: 29657.4160 - mse: 29657.4160 - mae: 171.3284 - val_loss: 31788.8379 - val_mse: 31788.8379 - val_mae: 178.2942
Epoch 143/180
65/65 [==============================] - 37s 565ms/step - loss: 29941.1562 - mse: 29941.1562 - mae: 172.1342 - val_loss: 23384.4590 - val_mse: 23384.4590 - val_mae: 152.9198
Epoch 144/180
65/65 [==============================] - 43s 657ms/step - loss: 29365.9590 - mse: 29365.9590 - mae: 170.7015 - val_loss: 30376.7754 - val_mse: 30376.7754 - val_mae: 174.2893
Epoch 145/180
65/65 [==============================] - 40s 608ms/step - loss: 29845.9277 - mse: 29845.9277 - mae: 171.9855 - val_loss: 33766.7031 - val_mse: 33766.7031 - val_mae: 183.7572
Epoch 146/180
65/65 [==============================] - 39s 606ms/step - lo

In [9]:
results = pd.DataFrame({'all_param':all_param,
                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})

results.to_csv('Results_LSTM_4_kernel_bias.csv')

y_pred = pd.DataFrame({'all_param': all_param,
                       'Predicitons': y_pred_list})

y_pred.to_csv('Pedictions_LSTM_4_kernel_bias.csv')

results

,all_param,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
0,{'bias_initializer': <tensorflow.python.ops.in...,99.690302,93.814721,96.021883,91.317437,100.223807,94.185704
1,{'bias_initializer': <tensorflow.python.ops.in...,33798.251775,33798.235473,33794.808429,33794.795412,33798.763269,33798.746509
2,{'bias_initializer': <tensorflow.python.ops.in...,61772.781259,61772.772340,61776.219521,61776.212400,61772.270474,61772.261304
3,{'bias_initializer': <tensorflow.python.ops.in...,167393.084225,167393.080933,167396.523622,167396.520994,167392.573282,167392.569897
4,{'bias_initializer': <tensorflow.python.ops.in...,213763.156019,213763.153442,213759.715439,213759.713381,213763.667128,213763.664478
5,{'bias_initializer': <tensorflow.python.ops.in...,275594.192308,275594.190308,275597.631965,275597.630369,275593.681328,275593.679272


In [10]:
results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})

def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
0,99.690302,93.814721,96.021883,91.317437,100.223807,94.185704
1,33798.251775,33798.235473,33794.808429,33794.795412,33798.763269,33798.746509
2,61772.781259,61772.772340,61776.219521,61776.212400,61772.270474,61772.261304
3,167393.084225,167393.080933,167396.523622,167396.520994,167392.573282,167392.569897
4,213763.156019,213763.153442,213759.715439,213759.713381,213763.667128,213763.664478
5,275594.192308,275594.190308,275597.631965,275597.630369,275593.681328,275593.679272
